# Part 1

I think the easiest way to do this challenge is to render the entire map.
The problem says the map is at least 1,000 grid units, so we have at least
a million values to store, which should be feasible.

Each cell in the map stores a list of IDs which have claimed it.

The only hitch is that we don't know how big the map needs to be in advance.
I tried writing a resizable map, but I think it's easier just to run through
the input twice: on the first pass, keep track of the maximum dimensions.
Then do the count on the second pass.

In [1]:
from IPython.display import display

In [53]:
def print_map(map_):
    ''' Render a map. Helpful for debugging. '''
    render = ''
    for row in map_:
        for val in row:
            if len(val) == 0:
                render += '·'
            elif len(val) == 1:
                render += str(val[0])[0]
            else:
                render += '*'
        render += '\n'
    print(render)

In [41]:
test_map = [
    [[], [1], [1], []],
    [[], [1], [1], []],
    [[2], [1,2], [1,2], [2]],
    [[2], [1,2], [1,2], [2]],
    [[], [1],  [1],  []],
]

print_map(test_map)

·11·
·11·
2**2
2**2
·11·



In [45]:
def init_map(rows, cols):
    map_ = list()
    for r in range(rows):
        map_.append(list())
        for col in range(cols):
            map_[r].append([])
    return map_

In [46]:
display(init_map(3, 2))

[[[], []], [[], []], [[], []]]

In [47]:
print_map(init_map(4, 5))

·····
·····
·····
·····



In [77]:
import re

# A claim looks like "#1 @ 704,926: 5x4".
claim_re = re.compile(r'#(\d+) @ (\d+),(\d+): (\d+)x(\d+)')

def parse_claim(claim):
    ''' Parse a line of text into a claim. 
    Note that the row/col and width/height are reversed. '''
    m = claim_re.match(claim)
    if not m:
        raise ValueError('Cannot parse claim: ' + claim)
    return int(m.group(1)), int(m.group(3)), int(m.group(2)), int(m.group(5)), int(m.group(4))

In [78]:
print(parse_claim("#1 @ 704,926: 5x4"))

(1, 926, 704, 4, 5)


In [79]:
print(parse_claim("#60 @ 835,405: 10x19"))

(60, 405, 835, 19, 10)


In [80]:
import itertools

def mark_claim(map_, id_, row, col, height, width):
    ''' Increment the squares on the map corresponding to the claim. '''
    for h, w in itertools.product(range(height), range(width)):
        map_[row + h][col + w].append(id_)

In [81]:
test_map = init_map(10, 10)
mark_claim(test_map, 1, 2, 3, 4, 5)
print_map(test_map)

··········
··········
···11111··
···11111··
···11111··
···11111··
··········
··········
··········
··········



In [82]:
mark_claim(test_map, 2, 4, 6, 4, 4)
print_map(test_map)

··········
··········
···11111··
···11111··
···111**22
···111**22
······2222
······2222
··········
··········



In [83]:
def count_map(map_, threshold=2):
    ''' Count the number of squares in the map greater than threshold. '''
    count = 0
    for row in map_:
        for val in row:
            if len(val) >= threshold:
                count += 1
    return count

In [84]:
count_map(test_map, 1)

32

In [85]:
count_map(test_map, 2)

4

In [86]:
# Try the test case.
test = [
    '#1 @ 1,3: 4x4\n',
    '#2 @ 3,1: 4x4\n',
    '#3 @ 5,5: 2x2\n',
]

# First pass
max_row = 0
max_col = 0

for line in test:
    id_, row, col, height, width = parse_claim(line)
    max_row = max(max_row, row + height)
    max_col = max(max_col, col + width)

print(max_row, max_col)

# Second pass
map_ = init_map(max_row, max_col)

for line in test:
    id_, row, col, height, width = parse_claim(line)
    mark_claim(map_, id_, row, col, height, width)

print_map(map_)
count_map(map_, 2)

7 7
·······
···2222
···2222
·11**22
·11**22
·111133
·111133



4

In [90]:
# Try another test case.
test = [
    '#1 @ 1,3: 4x4\n',
    '#2 @ 3,1: 4x4\n',
    '#3 @ 5,5: 2x2\n',
    '#4 @ 1,1: 3x3\n',
    '#5 @ 4,6: 4x2\n',
]

# First pass
max_row = 0
max_col = 0

for line in test:
    id_, row, col, height, width = parse_claim(line)
    max_row = max(max_row, row + height)
    max_col = max(max_col, col + width)

print(max_row, max_col)

# Second pass
map_ = init_map(max_row, max_col)

for line in test:
    id_, row, col, height, width = parse_claim(line)
    mark_claim(map_, id_, row, col, height, width)

print_map(map_)
count_map(map_, 2)

8 8
········
·44*222·
·44*222·
·****22·
·11**22·
·111133·
·111***5
····5555



11

In [91]:
# First pass: figure out the map's size.
max_row = 0
max_col = 0

with open('input.txt') as input_:
    for line in input_:
        id_, row, col, height, width = parse_claim(line)
        max_row = max(max_row, row + height)
        max_col = max(max_col, col + width)

print(max_row, max_col)

1000 999


In [93]:
# Second pass: mark claims on the map
map_ = init_map(max_row, max_col)

with open('input.txt') as input_:
    for line in input_:
        id_, row, col, height, width = parse_claim(line)
        mark_claim(map_, id_, row, col, height, width)

count_map(map_, 2)

116920

# Part 2

In this part, I can reuse the map I just made. Make a set of all claim IDs, then
iterate over the map. If a cell is claimed by 2 or more IDs, then remove all of 
those claiming IDs from the set.

In [94]:
ids = set()

with open('input.txt') as input_:
    for line in input_:
        id_, _, _, _, _ = parse_claim(line)
        ids.add(id_)

len(ids)

1359

In [ ]:
for row in map_:
    for val in row:
        if len(val) >= 2:
            ids -